In [10]:
from ortools.sat.python import cp_model
import pandas as pd

In [13]:
#debug reasons:
# variaveis_a_logar =[]
# def loga(variavel):
#     global variaveis_a_logar
#     variaveis_a_logar.append(variavel)




def resolve(solucionador, modelo, X):
    status = solucionador.Solve(modelo)
    print(f"Status: {status}")

    if status == cp_model.OPTIMAL:
        print("Optimal")
        for i, matriz_estacio_tempo in enumerate(X):
            for j, tempos in enumerate(matriz_estacio_tempo):
                for k, variavel in enumerate(tempos):
                    valor = solucionador.Value(variavel)
                    if valor == 1:
                        print(f"O aviao {i} no estacionamento {j} no tempo {k}")

    else:
        print("Solução não encontrada")    


In [29]:
modelo= cp_model.CpModel()
solucionador = cp_model.CpSolver()

total_avioes = 3
total_estacios= 2 
total_tempo =5
momento_de_chegada = [1,2,0]
tempo_druacao = [3,3,2]

X = [[[modelo.NewBoolVar(f"avioa_{i}_estacionamento_{j}_no_tempo{k}") for k in range(total_tempo)] for j in range(total_estacios)] for i in range(total_avioes)]

custos = [
    [1000, 500],
    [1000, 500],
    [1000, 500]
]

Y= []

#Restriçoes:

for i in range(total_avioes):
    posicoes_do_aviao = [X[i][j][k] for j in range(total_estacios) for k in range(total_tempo)]
    modelo.Add(sum(posicoes_do_aviao) >= 1)

for j in range(total_estacios):
    for k in range(total_tempo):
        avioes_agora = [X[i][j][k] for i in range(total_avioes)]
        modelo.Add(sum(avioes_agora) <= 1)

for i in range(total_avioes):
    linha_aviao_i = []
    for j in range(total_estacios):

        aviao_i_em_j = modelo.NewBoolVar(f"aviao_{i}_estacionamento_{j}")
        linha_aviao_i.append(aviao_i_em_j)
        variaveis_aviao_estacionamento = [X[i][j][k] for k in range(total_tempo)]

        modelo.Add(sum(variaveis_aviao_estacionamento) > 0).OnlyEnforceIf(aviao_i_em_j)        
        modelo.Add(sum(variaveis_aviao_estacionamento) == 0).OnlyEnforceIf(aviao_i_em_j.Not()) 

        aviao_i_em_outro_sem_ser_j = modelo.NewBoolVar(f"aviao_{i}_em_outro_sem_ser_{j}")       
        variaveis_outros_estacios= [X[i][outro_estacio][k] for k in range(total_tempo) for outro_estacio in range(total_estacios) if outro_estacio != j]

        modelo.Add(sum(variaveis_outros_estacios) > 0).OnlyEnforceIf(aviao_i_em_outro_sem_ser_j)
        modelo.Add(sum(variaveis_outros_estacios) == 0).OnlyEnforceIf(aviao_i_em_outro_sem_ser_j.Not())

        modelo.AddImplication(aviao_i_em_j, aviao_i_em_outro_sem_ser_j.Not())
    Y.append(linha_aviao_i)    

for i in range(total_avioes):
    tudo_aviao = [X[i][j][k] for j in range(total_estacios) for k in range(total_tempo)]
    modelo.Add(sum(tudo_aviao) == tempo_druacao[i])

for i in range(total_avioes):
    for j in range(total_estacios):
        for k in range(total_tempo):
            if k> 0 and total_tempo - 1:
                se_aviao_decolou = modelo.NewBoolVar(f"aviao_{i}_est_{j}_tempo{k}_decolou_agora")
                modelo.Add(sum([X[i][j][k-1], X[i][j][k].Not()]) == 2).OnlyEnforceIf(se_aviao_decolou)
                modelo.Add(sum([X[i][j][k-1], X[i][j][k].Not()]) != 2).OnlyEnforceIf(se_aviao_decolou.Not())

                for f in range(k+1, total_tempo):
                    modelo.AddImplication(se_aviao_decolou,X[i][j][f].Not())    

            if k < total_tempo - 1:
                se_aviao_pousou = modelo.NewBoolVar(f"aviao_{i}_est_{j}_tempo{k}_pousou_agora")
                modelo.Add(sum([X[i][j][k+1], X[i][j][k].Not()]) == 2).OnlyEnforceIf(se_aviao_pousou)
                modelo.Add(sum([X[i][j][k+1], X[i][j][k].Not()]) != 2).OnlyEnforceIf(se_aviao_pousou.Not())

                for p in range(0, k-1):
                    modelo.AddImplication(se_aviao_pousou,X[i][j][p].Not())

for i in range(total_avioes):
    momento_de_chegada_dele = momento_de_chegada[i]
    for j in range(total_estacios):
        for k in range(momento_de_chegada_dele):
            modelo.Add(X[i][j][k] == 0)

for i in range(total_avioes):
    momento_de_chegada_dele = momento_de_chegada[i]
    todos_os_estacios = [X[i][j][momento_de_chegada_dele] for j in range(total_estacios)]
    modelo.Add(sum(todos_os_estacios) == 1)

super_custo = [custos[i][j]*Y[i][j] for i in range(total_avioes) for j in range(total_estacios)]

modelo.Minimize(sum(super_custo))
resolve(solucionador, modelo, X)

Status: 4
Optimal
O aviao 0 no estacionamento 0 no tempo 1
O aviao 0 no estacionamento 0 no tempo 2
O aviao 0 no estacionamento 0 no tempo 3
O aviao 1 no estacionamento 1 no tempo 2
O aviao 1 no estacionamento 1 no tempo 3
O aviao 1 no estacionamento 1 no tempo 4
O aviao 2 no estacionamento 1 no tempo 0
O aviao 2 no estacionamento 1 no tempo 1
